In [3]:
import tensorflow as tf
import pandas as pd
from tqdm import tqdm
from scipy.signal import resample
from sklearn.model_selection import StratifiedKFold
import os
import numpy as np
import datetime
from tensorflow.keras.layers import Conv2D,Dense,Dropout,Input,GlobalMaxPooling2D,MaxPooling2D,LayerNormalization,BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
from sklearn.preprocessing import LabelEncoder
from scipy.stats import skew
from scipy.stats import kurtosis
from scipy.fftpack import fft,ifft
%matplotlib inline


Using TensorFlow backend.


In [20]:
os.chdir("/content/drive/My Drive/AI_Sensor")

In [21]:
semi_test=pd.read_csv("data/semi_test_8best.csv")
semi_test.sort_values(by=["proba"],ascending=False,inplace=True)
semi_test_len=len(semi_test)
print(semi_test_len)
semi_test.iloc[6750]

7500


fragment_id    2493.0000
behavior_id       5.0000
proba             0.5264
Name: 2493, dtype: float64

In [22]:
get_percentce=0.6
semi_test_percent=semi_test.iloc[:round(get_percentce*semi_test_len)]
sample_weight_train=semi_test_percent.proba.values
print(len(semi_test_percent))
sample_weight_train

4500


array([0.99999991, 0.99999988, 0.99999985, ..., 0.92402507, 0.92399134,
       0.92397419])

In [23]:
test = pd.read_csv('data/sensor_test.csv')
test.head()

,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg
0,0,71,0.2,1.0,0.6,0.2,5.3,9.2
1,0,150,0.0,1.0,-0.7,0.2,6.0,8.2
2,0,244,-0.2,0.8,-2.4,-0.4,5.3,7.6
3,0,326,2.3,-0.4,-1.6,2.5,4.5,3.4
4,0,409,-0.3,-1.4,3.3,0.2,3.5,12.4


In [24]:
test=test.merge(semi_test_percent,how="inner")
print(test.isna().any())
print(test.fragment_id.unique())
test.fragment_id=test.fragment_id+7500
test.head()

fragment_id    False
time_point     False
acc_x          False
acc_y          False
acc_z          False
acc_xg         False
acc_yg         False
acc_zg         False
behavior_id    False
proba          False
dtype: bool
[   0    1    4 ... 7494 7495 7498]


,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg,behavior_id,proba
0,7500,71,0.2,1.0,0.6,0.2,5.3,9.2,12,0.963707
1,7500,150,0.0,1.0,-0.7,0.2,6.0,8.2,12,0.963707
2,7500,244,-0.2,0.8,-2.4,-0.4,5.3,7.6,12,0.963707
3,7500,326,2.3,-0.4,-1.6,2.5,4.5,3.4,12,0.963707
4,7500,409,-0.3,-1.4,3.3,0.2,3.5,12.4,12,0.963707


In [25]:
test.drop("proba",axis=1,inplace=True)
print(len(test))
test.head()

257781


,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg,behavior_id
0,7500,71,0.2,1.0,0.6,0.2,5.3,9.2,12
1,7500,150,0.0,1.0,-0.7,0.2,6.0,8.2,12
2,7500,244,-0.2,0.8,-2.4,-0.4,5.3,7.6,12
3,7500,326,2.3,-0.4,-1.6,2.5,4.5,3.4,12
4,7500,409,-0.3,-1.4,3.3,0.2,3.5,12.4,12


In [26]:
train = pd.read_csv('data/sensor_train.csv')
print(len(train))
train=pd.concat([train,test])
print(len(train))
train.head()

425359
683140


,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg,behavior_id
0,0,27,0.3,-0.3,0.1,0.6,4.5,8.8,0
1,0,108,0.1,-0.0,-0.4,0.4,4.7,8.4,0
2,0,198,0.1,0.0,0.3,0.9,4.6,9.0,0
3,0,297,0.1,-0.1,-0.5,0.8,4.7,7.2,0
4,0,388,0.1,0.2,0.6,0.9,4.7,8.9,0


In [27]:
train_fragment_id=train.fragment_id.unique()

In [28]:
#train = pd.read_csv('data/sensor_train.csv')
test = pd.read_csv('data/sensor_test.csv')
sub = pd.read_csv('data/提交结果示例.csv')
mapping = {0: 'A_0', 1: 'A_1', 2: 'A_2', 3: 'A_3', 
        4: 'D_4', 5: 'A_5', 6: 'B_1',7: 'B_5', 
        8: 'B_2', 9: 'B_3', 10: 'B_0', 11: 'A_6', 
        12: 'C_1', 13: 'C_3', 14: 'C_0', 15: 'B_6', 
        16: 'C_2', 17: 'C_5', 18: 'C_6'}
mapping3 = {'A':0, 'B':1, 'C':2, 'D':2}

### 处理标签数据
train['action_type'] = train['behavior_id'].map(mapping)
train['action_type_1'] = train['action_type'].map(lambda x:x.split('_')[0])
train['action_type_2'] = train['action_type'].map(lambda x:int(x.split('_')[1]))
train['sence'] = train['behavior_id'].map(lambda x:mapping3[mapping[x].split('_')[0]])
train['action'] = train['behavior_id'].map(lambda x:int(mapping[x].split('_')[1]))
y_cls = train.groupby('fragment_id')['behavior_id'].min()
y_scene = train.groupby('fragment_id')['sence'].min()
y_action = train.groupby('fragment_id')['action'].min()

sample_weight_train=np.concatenate([[1]*(len(y_cls)-len(sample_weight_train)),sample_weight_train])
print("label",len(y_cls))
print("sample_weight_train",len(sample_weight_train))

group_feature=pd.read_csv('data/group_feature.csv')
group_feature=group_feature.drop(r'Unnamed: 0',axis=1)
print(group_feature[group_feature.fragment_id==1])
print(train.head())
group_feature.head()

label 11792
sample_weight_train 11792
   fragment_id  behavior_id  ...  accmodg_fft_maxind  accmodg_fft_minind
1            1          0.0  ...                26.0                13.0

[1 rows x 298 columns]
   fragment_id  time_point  acc_x  ...  action_type_2  sence  action
0            0          27    0.3  ...              0      0       0
1            0         108    0.1  ...              0      0       0
2            0         198    0.1  ...              0      0       0
3            0         297    0.1  ...              0      0       0
4            0         388    0.1  ...              0      0       0

[5 rows x 14 columns]


,fragment_id,behavior_id,acc_x_time_sum,acc_x_time_mean,acc_x_time_std,acc_x_time_var,acc_x_time_max,acc_x_time_min,acc_x_time_median,acc_x_time_energy,acc_x_time_mad,acc_x_time_percent_9,acc_x_time_percent_75,acc_x_time_percent_25,acc_x_time_percent_1,acc_x_time_percent_75_25,acc_x_time_range,acc_x_time_zcr,acc_x_time_mcr,acc_x_time_minind,acc_x_time_maxind,acc_x_time_skew,acc_x_time_kurtosis,acc_x_time_zero_big,acc_x_time_zero_small,acc_x_time_len,acc_x_fft_dc,acc_x_fft_mean,acc_x_fft_var,acc_x_fft_std,acc_x_fft_sum,acc_x_fft_entropy,acc_x_fft_energy,acc_x_fft_skew,acc_x_fft_kurtosis,acc_x_fft_max,acc_x_fft_min,acc_x_fft_maxind,acc_x_fft_minind,acc_y_time_sum,...,accmod_fft_min,accmod_fft_maxind,accmod_fft_minind,accmodg_time_sum,accmodg_time_mean,accmodg_time_std,accmodg_time_var,accmodg_time_max,accmodg_time_min,accmodg_time_median,accmodg_time_energy,accmodg_time_mad,accmodg_time_percent_9,accmodg_time_percent_75,accmodg_time_percent_25,accmodg_time_percent_1,accmodg_time_percent_75_25,accmodg_time_range,accmodg_time_zcr,accmodg_time_mcr,accmodg_time_minind,accmodg_time_maxind,accmodg_time_skew,accmodg_time_kurtosis,accmodg_time_zero_big,accmodg_time_zero_small,accmodg_time_len,accmodg_fft_dc,accmodg_fft_mean,accmodg_fft_var,accmodg_fft_std,accmodg_fft_sum,accmodg_fft_entropy,accmodg_fft_energy,accmodg_fft_skew,accmodg_fft_kurtosis,accmodg_fft_max,accmodg_fft_min,accmodg_fft_maxind,accmodg_fft_minind
0,0,0.0,-0.1,-0.001754,0.192391,0.037014,0.4,-0.9,0.0,2.11,0.118190,-0.6480,-0.69000,-0.83000,-0.8720,0.2,1.3,30.0,22.0,14.0,8.0,-1.482822,6.896239,18.0,18.0,57.0,0.1,1.292198,0.477724,0.691176,36.181548,141.280517,60.13,0.469282,-0.825276,2.721748,0.280996,23.0,10.0,-0.1,...,0.029661,3.0,19.0,557.721098,9.784581,0.290883,0.084613,10.560776,8.635392,9.786726,5461.89,0.202343,8.933630,8.883924,8.718236,8.668530,0.282354,1.925384,0.0,40.0,3.0,14.0,-0.659187,3.638172,57.0,0.0,57.0,557.721098,1.903873,1.284328,1.133282,53.308437,142.393179,137.453666,0.802231,-0.280091,4.520294,0.323936,25.0,17.0
1,1,0.0,-1.4,-0.025000,0.108972,0.011875,0.2,-0.5,0.0,0.70,0.081250,-0.3515,-0.37625,-0.45875,-0.4835,0.1,0.7,34.0,25.0,47.0,34.0,-1.314397,4.567471,13.0,21.0,56.0,1.4,0.769447,0.084379,0.290481,21.544527,138.217262,18.94,-0.076908,-0.632877,1.341267,0.243630,23.0,10.0,1.5,...,0.053676,12.0,14.0,549.898472,9.819616,0.200375,0.040150,10.521407,9.441398,9.791067,5402.04,0.147439,9.480556,9.474029,9.452275,9.445749,0.194722,1.080009,0.0,23.0,36.0,52.0,1.024599,1.759842,56.0,0.0,56.0,549.898472,1.343474,0.488727,0.699090,37.617259,141.492146,64.222154,0.355272,-0.498603,3.033059,0.275922,26.0,13.0
2,2,0.0,-1.6,-0.028070,0.237497,0.056405,0.7,-0.5,0.0,3.26,0.172853,-0.5000,-0.50000,-0.50000,-0.5000,0.3,1.2,32.0,26.0,31.0,46.0,0.476711,1.053196,17.0,24.0,57.0,1.6,1.640215,0.582196,0.763018,45.926007,141.897584,91.63,-0.143329,-0.607145,3.174333,0.137414,12.0,2.0,-0.3,...,0.776809,0.0,23.0,558.066365,9.790638,0.352292,0.124110,10.813418,8.823831,9.827004,5470.90,0.236079,8.846336,8.842585,8.830083,8.826332,0.217304,1.989586,0.0,35.0,46.0,45.0,-0.199623,1.722709,57.0,0.0,57.0,558.066365,2.429942,1.295966,1.138405,68.038382,139.858288,201.616388,0.447226,-0.018656,5.442073,0.619054,9.0,2.0
3,3,0.0,0.1,0.001818,0.177345,0.031451,0.5,-0.4,0.0,1.73,0.126017,-0.4000,-0.40000,-0.40000,-0.4000,0.2,0.9,35.0,20.0,14.0,17.0,0.070294,0.777394,19.0,19.0,55.0,0.1,1.174887,0.381493,0.617651,31.721943,135.085597,47.57,0.430435,-0.648286,2.422155,0.193962,26.0,4.0,1.3,...,0.552758,1.0,10.0,537.191665,9.767121,0.357362,0.127707,10.585367,8.828363,9.780082,5253.84,0.271757,8.957173,8.935705,8.864144,8.842676,0.467731,1.757004,0.0,35.0,46.0,45.0,0.188511,0.325010,55.0,0.0,55.0,537.191665,2.302681,1.851639,1.360750,62.172377,136.760997,193.157392,0.856965,0.349557,5.952026,0.176151,21.0,3.0
4,4,0.0,0.1,0.001818,0.334931,0.112179,0.9,-1.3,0.0,6.17,0.238413,-1.0084,-1.05700,-1.21900,-1.2676,0.4,2.2,37.0,30.0,18.0,32.0,-0.741681,3.131766,24.0,21.0,55.0,0.1,2.256830,1.

In [29]:
train.head()

,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg,behavior_id,action_type,action_type_1,action_type_2,sence,action
0,0,27,0.3,-0.3,0.1,0.6,4.5,8.8,0,A_0,A,0,0,0
1,0,108,0.1,-0.0,-0.4,0.4,4.7,8.4,0,A_0,A,0,0,0
2,0,198,0.1,0.0,0.3,0.9,4.6,9.0,0,A_0,A,0,0,0
3,0,297,0.1,-0.1,-0.5,0.8,4.7,7.2,0,A_0,A,0,0,0
4,0,388,0.1,0.2,0.6,0.9,4.7,8.9,0,A_0,A,0,0,0


In [30]:
train['accmod'] = (train.acc_x ** 2 + train.acc_y ** 2 + train.acc_z ** 2) ** .5
train['accmodg'] = (train.acc_xg ** 2 + train.acc_yg ** 2 + train.acc_zg ** 2) ** .5
#train['acc_x_time'] = train.acc_x*np.abs((train['time_point']-5000))/np.log(5e4)
#train['acc_y_time'] = train.acc_y*np.abs((train['time_point']-5000))/np.log(5e4)
#train['acc_z_time'] = train.acc_z*np.abs((train['time_point']-5000))/np.log(5e4)
#train['acc_xg_time'] = train.acc_xg*np.abs((train['time_point']-5000))/np.log(5e4)
#train['acc_yg_time'] = train.acc_yg*np.abs((train['time_point']-5000))/np.log(5e4)
#train['acc_zg_time'] = train.acc_zg*np.abs((train['time_point']-5000))/np.log(5e4)
#train['modtime'] = (train.acc_x ** 2 + train.acc_y ** 2 + train.acc_z ** 2) ** .5*np.abs((train['time_point']-5000))/np.log(5e4)
#train['modgtime'] = (train.acc_xg ** 2 + train.acc_yg ** 2 + train.acc_zg ** 2) ** .5*np.abs((train['time_point']-5000))/np.log(5e4)
#train['xy'] = (train['acc_x'] ** 2 + train['acc_y'] ** 2) ** 0.5
#train['xy_g'] = (train['acc_xg'] ** 2 + train['acc_yg'] ** 2) ** 0.5

test['accmod'] = (test.acc_x ** 2 + test.acc_y ** 2 + test.acc_z ** 2) ** .5
test['accmodg'] = (test.acc_xg ** 2 + test.acc_yg ** 2 + test.acc_zg ** 2) ** .5                                      
#test['acc_x_time'] = test.acc_x*np.abs((test['time_point']-5000))/np.log(5e4)
#test['acc_y_time'] = test.acc_y*np.abs((test['time_point']-5000))/np.log(5e4)
#test['acc_z_time'] = test.acc_z*np.abs((test['time_point']-5000))/np.log(5e4)
#test['acc_xg_time'] = test.acc_xg*np.abs((test['time_point']-5000))/np.log(5e4)
#test['acc_yg_time'] = test.acc_yg*np.abs((test['time_point']-5000))/np.log(5e4)
#test['acc_zg_time'] = test.acc_zg*np.abs((test['time_point']-5000))/np.log(5e4) 
#test['modtime'] = (test.acc_x ** 2 + test.acc_y ** 2 + test.acc_z ** 2) ** .5*np.abs((test['time_point']-5000))/np.log(5e4)
#test['modgtime'] = (test.acc_xg ** 2 + test.acc_yg ** 2 + test.acc_zg ** 2) ** .5*np.abs((test['time_point']-5000))/np.log(5e4)   
#test['xy'] = (test['acc_x'] ** 2 + test['acc_y'] ** 2) ** 0.5
#test['xy_g'] = (test['acc_xg'] ** 2 + test['acc_yg'] ** 2) ** 0.5


In [31]:
x = np.zeros((len(train_fragment_id), 60, 8, 1))  
#x = np.zeros((7292, 60, 8, 1))
t = np.zeros((7500, 60, 8, 1))
for i in tqdm(range(len(train_fragment_id))):
    tmp = train[train.fragment_id ==train_fragment_id[i]][:60]
    x[i,:,:, 0] = resample(tmp.drop(['fragment_id', 'time_point', 'behavior_id','action_type','action_type_1','action_type_2',"sence","action"],axis=1), 60, np.array(tmp.time_point))[0]
for i in tqdm(range(7500)):
    tmp = test[test.fragment_id == i][:60]
    t[i,:,:, 0] = resample(tmp.drop(['fragment_id', 'time_point'],axis=1), 60, np.array(tmp.time_point))[0]


100%|██████████| 7500/7500 [00:18<00:00, 409.11it/s]


In [32]:
def jitter(x,snr_db):
    """
    根据信噪比添加噪声
    :param x:
    :param snr_db:
    :return:
    """
    # 随机选择信噪比
    assert isinstance(snr_db, list)
    snr_db_low = snr_db[0]
    snr_db_up = snr_db[1]
    snr_db = np.random.randint(snr_db_low, snr_db_up, (1,))[0]

    snr = 10 ** (snr_db / 10)
    Xp = np.sum(x ** 2, axis=0, keepdims=True) / x.shape[0]  # 计算信号功率
    Np = Xp / snr  # 计算噪声功率
    n = np.random.normal(size=x.shape, scale=np.sqrt(Np), loc=0.0)  # 计算噪声 loc均值，scale方差
    xn = x + n
    return xn

def standardization(X):
    #X batch_size 60 8 1
    x1 = X.transpose(0, 1, 3, 2)
    #x1 = X

    x2 = x1.reshape(-1, x1.shape[-1])
    print(x2.shape)
    mean = [8.03889039e-03, -6.41381949e-02, 2.37856977e-02, 8.64949391e-01,
            2.80964889e+00, 7.83041714e+00, 6.44853358e-01, 9.78580749e+00]
    std = [0.6120893, 0.53693888, 0.7116134, 3.22046385, 3.01195336, 2.61300056, 0.87194132, 0.68427254]
    mu=np.array(mean)
    sigma=np.array(std)
    x3 = ((x2 - mu) / (sigma))
    print(x3.shape)
    x4 = x3.reshape(x1.shape).transpose(0, 1, 3, 2)
    #x4 = x3.reshape(x1.shape)
    return x4


In [33]:
#标准化
x=standardization(x)
t=standardization(t)
#sample_weight_train=np.concatenate([sample_weight_train, sample_weight_train], axis=0)
#sample_weight_train.shape

(707520, 8)
(707520, 8)
(450000, 8)
(450000, 8)


In [34]:
x1= jitter(x,[5,15])
import random
def shuffle_index_func(index):
  index=random.shuffle(index)
  return index
def mixup(x,y_cls,y_scene,y_action,random_max=0.08):
  shuffle_index=list(range(x.shape[0]))
  r_index1=shuffle_index_func(shuffle_index)
  r_index2=shuffle_index_func(shuffle_index)
  rd=random.uniform(0.03,random_max)
  batch_x_mixup=(1-rd)*x[r_index1]+rd*x[r_index2]
  batch_y_cls_mixup=y_cls[r_index1]
  batch_y_scene_mixup=y_scene[r_index1]
  batch_y_action_mixup=y_action[r_index1]
  #batch_y_mixup=(1-rd)*y[r_index1]+rd*y[r_index2]
  return batch_x_mixup,batch_y_cls_mixup,batch_y_scene_mixup,batch_y_action_mixup
x2,y_cls2,y_scene2,y_action2=mixup(x,y_cls.values,y_scene.values,y_action.values,random_max=0.08)
x = np.concatenate([x,x1,x2[0]], axis=0)
y_cls = np.concatenate([y_cls,y_cls,y_cls2[0]], axis=0)
y_scene = np.concatenate([y_scene,y_scene,y_scene2[0]], axis=0)
y_action = np.concatenate([y_action,y_action,y_action2[0]], axis=0)
print(x.shape,y_cls.shape,y_scene.shape,y_action.shape)

(35376, 60, 8, 1) (35376,) (35376,) (35376,)


In [35]:
class Disout(tf.keras.layers.Layer):
    '''
    disout
    论文：https://arxiv.org/abs/2002.11022
    '''
    def __init__(self, dist_prob, block_size=5, alpha=1, **kwargs):
        super(Disout, self).__init__(**kwargs)
        self.dist_prob = dist_prob
        self.weight_behind=None

        self.alpha = alpha
        self.block_size = block_size

    def build(self, input_shape):
        pass

    @tf.function
    def call(self, x):
        '''x：(batch_size,h,w,c)'''
        if not self.trainable:
            return x
        else:
            if tf.math.equal(tf.rank(x),4):
                x_shape = tf.shape(x)
                x_size = x_shape[1:3]
                x_size_f = tf.cast(x_size, tf.float32)
                # 计算block_size
                x_block_size_f = tf.constant((self.block_size, self.block_size), tf.float32)
                # x_block_size_f = x_size_f * self.block_size
                # x_block_size_f = tf.math.maximum(x_block_size_f, 1)
                x_block_size = tf.cast(x_block_size_f, tf.int32)
                # 根据dist_prob，计算block_num
                x_block_num = (x_size_f[0] * x_size_f[1]) * self.dist_prob / (x_block_size_f[0] * x_block_size_f[1])
                # 计算block在中心区域出现的概率
                x_block_rate = x_block_num / ((x_size_f[0] - x_block_size_f[0] + 1) * (x_size_f[1] - x_block_size_f[1] + 1))
                # tf.print('x_block_rate:', x_block_rate)
                # 根据概率生成block区域
                x_block_center = tf.random.uniform((x_shape[0], x_size[0] - x_block_size[0] + 1, x_size[1] - x_block_size[1] + 1, x_shape[3]), dtype=tf.float32)
                x_block_padding_t = x_block_size[0] // 2
                x_block_padding_b = x_size_f[0] - tf.cast(x_block_padding_t, tf.float32) - (x_size_f[0] - x_block_size_f[0] + 1.0)
                x_block_padding_b = tf.cast(x_block_padding_b, tf.int32)
                x_block_padding_l = x_block_size[1] // 2
                x_block_padding_r = x_size_f[1] - tf.cast(x_block_padding_l, tf.float32) - (x_size_f[1] - x_block_size_f[1] + 1.0)
                x_block_padding_r = tf.cast(x_block_padding_r, tf.int32)
                x_block_padding = tf.pad(x_block_center,[[0, 0],[x_block_padding_t, x_block_padding_b],[x_block_padding_l, x_block_padding_r],[0, 0]])
                x_block = tf.cast(x_block_padding<x_block_rate, tf.float32)
                x_block = tf.nn.max_pool2d(x_block, ksize=[self.block_size, self.block_size], strides=[1, 1], padding='SAME')
                # block百分比
                # x_block_percent_ones = tf.reduce_sum(x_block) / tf.reduce_prod(tf.cast(tf.shape(x_block), tf.float32))
                # tf.print('x_block_percent_ones:', x_block_percent_ones, tf.shape(x_block))
                # 特征叠加
                x_abs = tf.abs(x)
                x_sum = tf.math.reduce_sum(x_abs, axis=-1, keepdims=True)
                x_max = tf.math.reduce_max(x_sum, axis=(1, 2), keepdims=True)
                x_max_c = tf.math.reduce_max(x_abs, axis=(1, 2), keepdims=True)
                x_sum_c = tf.math.reduce_sum(x_max_c, axis=-1, keepdims=True)
                x_v = x_sum / x_sum_c
                # tf.print('x_v:', tf.shape(x_v), tf.math.reduce_min(x_v), tf.math.reduce_max(x_v))
                # 特征方差
                # x_variance = tf.math.reduce_variance(x_sum, axis=(1, 2), keepdims=True)
                # tf.print('x_variance:', tf.shape(x_variance), tf.math.reduce_min(x_variance), tf.math.reduce_max(x_variance))
                # 叠加扰动
                x_max = tf.reduce_max(x, axis=(1,2), keepdims=True)
                x_min = tf.reduce_min(x, axis=(1,2), keepdims=True)
                x_block_random = tf.random.uniform(x_shape, dtype=x.dtype) * (x_max - x_min) + x_min
                x_block_random = x_block_random * (self.alpha * x_v + 0.3) + x * (1.0 - self.alpha * x_v - 0.3)
                x = x * (1-x_block) + x_block_random * x_block
                return x
            else:
                return x

    def compute_output_shape(self, input_shape):
        '''计算输出shape'''
        return input_shape
        
class Disout1D(tf.keras.layers.Layer):
    '''
    disout
    论文：https://arxiv.org/abs/2002.11022
    '''

    def __init__(self, dist_prob, block_size=5, alpha=0.5, **kwargs):
        super(Disout1D, self).__init__(**kwargs)
        self.dist_prob = dist_prob

        self.alpha = alpha
        self.block_size = block_size

    def build(self, input_shape):
        pass

    @tf.function
    def call(self, x):
        '''x：(batch_size,h,w,c)'''
        if not self.trainable:
            return x
        else:
            if tf.math.equal(tf.rank(x),2):
                x_shape = tf.shape(x)
                x_size = x_shape[1]
                x_size_f = tf.cast(x_size, tf.float32)
                # 计算block_size
                x_block_size_f = tf.constant(self.block_size, tf.float32)
                # x_block_size_f = x_size_f * self.block_size
                # x_block_size_f = tf.math.maximum(x_block_size_f, 1)
                x_block_size = tf.cast(x_block_size_f, tf.int32)
                # 根据dist_prob，计算block_num
                x_block_num = (x_size_f) * self.dist_prob / (x_block_size_f)
                # 计算block在中心区域出现的概率
                x_block_rate = x_block_num / ((x_size_f - x_block_size_f + 1))
                # 根据概率生成block区域
                x_block_center = tf.random.uniform((x_shape[0], x_size - x_block_size + 1), dtype=tf.float32)
                x_block_padding_t = x_block_size // 2
                x_block_padding_b = x_size_f - tf.cast(x_block_padding_t, tf.float32) - (x_size_f - x_block_size_f + 1.0)
                x_block_padding_b = tf.cast(x_block_padding_b, tf.int32)
                x_block_padding = tf.pad(x_block_center,[[0, 0],[x_block_padding_t, x_block_padding_b]])
                x_block = tf.cast(x_block_padding<x_block_rate, tf.float32)
                x_block = tf.expand_dims(x_block, axis=-1)
                x_block = tf.nn.max_pool1d(x_block, ksize=[self.block_size], strides=[1], padding='SAME')
                x_block = tf.reshape(x_block, x_shape)
                # 叠加扰动
                x_max = tf.reduce_max(x, axis=1, keepdims=True)
                x_min = tf.reduce_min(x, axis=1, keepdims=True)
                x_block_random = tf.random.uniform(x_shape, dtype=x.dtype) * (x_max - x_min) + x_min
                x_block_random = x_block_random * (1.0 - self.alpha) + x * self.alpha
                x = x * (1-x_block) + x_block_random * x_block
                return x
            else:
                return x

    def compute_output_shape(self, input_shape):
        '''计算输出shape'''
        return input_shape


In [79]:
def loss_multi(y_true, y_pred, smooth_wight=0.001, eps=0.001):
    X_cls, X_sence, X_action = y_pred[0], y_pred[1], y_pred[2]
    y_cls, y_sence, y_action = y_true[0], y_true[1], y_true[2]
    print(X_cls.shape,X_sence.shape,X_action.shape)
    print(y_cls.shape,y_cls.shape,y_cls.shape)
    print("test")
    loss_cls = tf.losses.categorical_crossentropy(y_cls, X_cls)
    loss_sence = tf.losses.categorical_crossentropy(y_sence, X_sence)
    loss_action = tf.losses.categorical_crossentropy(y_action, X_action)
    
#     return [loss_cls, loss_sence, loss_action]
    return tf.reduce_mean(1.0/7 * loss_sence + 1.0/3 * loss_action + 11/21 * loss_cls)

def get_acc_combo():
    def combo(y, y_pred):
        # 数值ID与行为编码的对应关系
        mapping = {0: 'A_0', 1: 'A_1', 2: 'A_2', 3: 'A_3',
            4: 'D_4', 5: 'A_5', 6: 'B_1',7: 'B_5',
            8: 'B_2', 9: 'B_3', 10: 'B_0', 11: 'A_6',
            12: 'C_1', 13: 'C_3', 14: 'C_0', 15: 'B_6',
            16: 'C_2', 17: 'C_5', 18: 'C_6'}
        # 将行为ID转为编码
        code_y, code_y_pred = mapping[int(y)], mapping[int(y_pred)]
        if code_y == code_y_pred: #编码完全相同得分1.0
            return 1.0
        elif code_y.split("_")[0] == code_y_pred.split("_")[0]: #编码仅字母部分相同得分1.0/7
            return 1.0/7
        elif code_y.split("_")[1] == code_y_pred.split("_")[1]: #编码仅数字部分相同得分1.0/3
            return 1.0/3
        else:
            return 0.0

    confusionMatrix=np.zeros((19,19))
    for i in range(19):
        for j in range(19):
            confusionMatrix[i,j]=combo(i,j)
    confusionMatrix=tf.convert_to_tensor(confusionMatrix)

    def acc_combo(y, y_pred):
        y=tf.argmax(y,axis=1)
        y_pred = tf.argmax(y_pred, axis=1)
        indices=tf.stack([y,y_pred],axis=1)#在1轴增加一个维度
        scores=tf.gather_nd(confusionMatrix,tf.cast(indices,tf.int32))
        return tf.reduce_mean(scores)
    return acc_combo

In [80]:
kfold = StratifiedKFold(5, shuffle=True)

In [81]:
def Net():
    input = Input(shape=(60,8,1))
    X = Conv2D(filters=64,
               kernel_size=3,
               activation='relu',
               padding='same')(input)
    X=BatchNormalization()(X)
    X = Conv2D(filters=128,
               kernel_size=3,
               activation='relu',
               padding='same')(X)
    X=BatchNormalization()(X)
    X = MaxPooling2D()(X)
    X = Conv2D(filters=256,
               kernel_size=3,
               activation='relu',
               padding='same')(X)
    X=BatchNormalization()(X)
    X = Conv2D(filters=512,
               kernel_size=3,
               activation='relu',
               padding='same')(X)
    X=BatchNormalization()(X)
    X = GlobalMaxPooling2D()(X)
    X = Dropout(0.3)(X)
    X_cls = Dense(19,name="cls",activation='softmax')(X)
    X_sence = Dense(4,name="sence",activation='softmax')(X)
    X_action = Dense(7,name="action",activation='softmax')(X)
    
    return tf.keras.Model([input], [X_cls,X_sence,X_action])

class CustomModel(tf.keras.Model):
    '''自定义模型'''
    def __init__(self):
        '''初始化模型层'''
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(
            filters=64, kernel_size=3,strides=1,padding='same',activation='relu')
        self.batchnorm1=tf.keras.layers.BatchNormalization()
        self.disout1 = Disout(0.3, block_size=3)


        self.conv2 = tf.keras.layers.Conv2D(
            filters=128, kernel_size=3,strides=1,padding='same',activation='relu')
        self.batchnorm2=tf.keras.layers.BatchNormalization()
        self.disout2 = Disout(0.3, block_size=3)
        self.pool1 = tf.keras.layers.MaxPool2D()

        self.conv3 = tf.keras.layers.Conv2D(
            filters=256, kernel_size=3,strides=1,padding='same',activation='relu')
        #,kernel_regularizer=tf.keras.regularizers.l1_l2(0.01)
        self.batchnorm3=tf.keras.layers.BatchNormalization()
        self.disout3 = Disout(0.3, block_size=3)
        self.pool2 = tf.keras.layers.MaxPool2D()

        self.conv4 = tf.keras.layers.Conv2D(
            filters=256, kernel_size=3,strides=1,padding='same',activation='relu')
        self.batchnorm4=tf.keras.layers.BatchNormalization()
        self.disout4 = Disout(0.3, block_size=3)

        self.flatten = tf.keras.layers.GlobalMaxPooling2D()
        self.disout5 = Disout1D(0.5, block_size=1)
        self.dropout= tf.keras.layers.Dropout(0.3)
        self.fc = tf.keras.layers.Dense(19, activation='softmax')

    def call(self, x):
        '''运算部分'''
        x=self.conv1(x)
        x=self.batchnorm1(x)
        #x=self.disout1(x)


        x=self.conv2(x)
        x=self.batchnorm2(x)
        #x=self.disout2(x)
        x=self.pool1(x)

        x=self.conv3(x)
        x=self.batchnorm3(x)
        #x=self.disout3(x)
        #x=self.pool2(x)

        x=self.conv4(x)
        x=self.batchnorm4(x)
        #x=self.disout4(x)

        x=self.flatten(x)
        x=self.dropout(x)
        x=self.fc(x)
        return x



In [82]:
import random
def shuffle_index_func(index):
  index=random.shuffle(index)
  return index

def shuffle_data(data_x,data_y):
  shuffle_index=list(range(data_x.shape[0]))
  r_index=shuffle_index_func(shuffle_index)
  return data_x[r_index],data_y[r_index]

def mixup(batch_x,batch_y,random_max=0.08):
  shuffle_index=list(range(batch_x.shape[0]))
  r_index1=shuffle_index_func(shuffle_index)
  r_index2=shuffle_index_func(shuffle_index)
  rd=random.uniform(0.03,random_max)
  batch_x_mixup=(1-rd)*batch_x[r_index1]+rd*batch_x[r_index2]
  batch_y_mixup=(1-rd)*batch_y[r_index1]+rd*batch_y[r_index2]
  return batch_x_mixup,batch_y_mixup

def noise(batch_x,batch_y,random_max=0.05):
  size=batch_x.shape
  batch_x_nosie=batch_x+np.random.uniform(-random_max,random_max,size=size)
  batch_y_noise=batch_y

  return batch_x_nosie,batch_y_noise

def multi_jitter(batch_x,batch_y,snr_db):
    """
    根据信噪比添加噪声
    :param batch_x:
    :param batch_x_stat:
    :param snr_db:
    :return:
    """
    # 随机选择信噪比
    assert isinstance(snr_db, list)
    snr_db_low = snr_db[0]
    snr_db_up = snr_db[1]
    snr_db = np.random.randint(snr_db_low, snr_db_up, (1,))[0]

    snr = 10 ** (snr_db / 10)
    Xp = np.sum(batch_x ** 2, axis=0, keepdims=True) / batch_x.shape[0]  # 计算信号功率
    Np = Xp / snr  # 计算噪声功率
    n = np.random.normal(size=batch_x.shape, scale=np.sqrt(Np), loc=0.0)  # 计算噪声 loc均值，scale方差
    xn = batch_x + n

    batch_y_n=batch_y

    return xn,batch_y_n

def genrator(data_x,data_y,batch_size,dataAugment=True,shuffle=False):
  #先把shuffle打乱
  if shuffle==True:
    data_x,data_y=shuffle_data(data_x,data_y)

  #产生batch
  while True:
    data_num=data_x.shape[0]
    range_num=data_num//batch_size+1
    for i in range(range_num):
      begin_index,end_index=i*batch_size,(i+1)*batch_size

      list_batch_x,list_batch_y=[],[]
      if begin_index<data_num:
        batch_x,batch_y=data_x[begin_index:end_index],data_y[begin_index:end_index]
        list_batch_x.append(batch_x),list_batch_y.append(batch_y)

        if dataAugment==True:

          #mixup
          batch_x_mixup,batch_y_mixup=mixup(batch_x,batch_y,random_max=0.08)
          list_batch_x.append(batch_x_mixup[0]),list_batch_y.append(batch_y_mixup[0])
          #print(batch_x_mixup[0].shape,batch_y_mixup[0].shape)

          #noise
          batch_x_noise,batch_y_noise=noise(batch_x,batch_y,random_max=0.05)
          list_batch_x.append(batch_x_noise),list_batch_y.append(batch_y_noise)
          #print(batch_x_noise.shape,batch_y_noise.shape)

          #jitter_noise
          batch_x_jitter,batch_y_jitter=multi_jitter(batch_x,batch_y,[5,15])
          list_batch_x.append(batch_x_jitter),list_batch_y.append(batch_y_jitter)
          #print(batch_x_jitter.shape,batch_y_jitter.shape)

          batch_x_yield,batch_y_yield=np.vstack(list_batch_x),np.vstack(list_batch_y)

          yield batch_x_yield[0],batch_y_yield[0]

In [83]:
proba_t = np.zeros((7500, 19))
for fold, (xx, yy) in enumerate(kfold.split(x, y_cls)):
    y_cls_ = tf.keras.utils.to_categorical(y_cls, num_classes=19)
    y_sence_ = tf.keras.utils.to_categorical(y_scene, num_classes=4)
    y_action_ = tf.keras.utils.to_categorical(y_action, num_classes=7)
    #model=CustomModel()
    model = Net()
    model.compile(loss=loss_multi,
                  #"categorical_crossentropy"
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  metrics=['acc',get_acc_combo()])
    plateau = tf.keras.callbacks.ReduceLROnPlateau(monitor="cls_acc",
                                verbose=1,
                                mode='max',
                                factor=0.1,
                                patience=3)
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='cls_acc',
                                   verbose=1,
                                   mode='max',
                                   patience=20)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(f'fold{fold}.h5',
                                 monitor='cls_acc',
                                 verbose=1,
                                 mode='max',
                                 save_best_only=True)
    gen_data_true=genrator(x[xx], y_cls_[xx],32,dataAugment=True,shuffle=True)
    gen_data_false=genrator(x[xx], y_cls_[xx],32,dataAugment=False,shuffle=True)
    model.summary()
    model.fit(#gen_data_true,
              x[xx],[y_cls_[xx],y_sence_[xx],y_action_[xx]],
              batch_size=64,
              epochs=150,
              #steps_per_epoch=x[xx].shape[0]//32+1,
              #sample_weight=sample_weight_train[xx],
              verbose=1,
              shuffle=True,
              validation_data=[x[yy], [y_cls_[yy],y_sence_[yy],y_action_[yy]]],
              callbacks=[plateau, early_stopping, checkpoint])
    model.load_weights(f'fold{fold}.h5')
    predict= model.predict(t, verbose=1, batch_size=1024) / 5.
    proba_t+=predict[0]

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 60, 8, 1)]   0                                            
__________________________________________________________________________________________________
conv2d_44 (Conv2D)              (None, 60, 8, 64)    640         input_12[0][0]                   
__________________________________________________________________________________________________
batch_normalization_44 (BatchNo (None, 60, 8, 64)    256         conv2d_44[0][0]                  
__________________________________________________________________________________________________
conv2d_45 (Conv2D)              (None, 60, 8, 128)   73856       batch_normalization_44[0][0]     
___________________________________________________________________________________________

KeyboardInterrupt: ignored

In [ ]:
#model=CustomModel()
#print(model.summary())
#model.load_weights('fold3.h5',by_name=True)
proba_y = np.zeros((len(x), 19))
predict=model.predict(x, verbose=1, batch_size=1024)
proba_y=np.argmax(predict[0],axis=1)
#print(proba_y)
def acc_combo(y, y_pred):
    # 数值ID与行为编码的对应关系
    mapping = {0: 'A_0', 1: 'A_1', 2: 'A_2', 3: 'A_3', 
        4: 'D_4', 5: 'A_5', 6: 'B_1',7: 'B_5', 
        8: 'B_2', 9: 'B_3', 10: 'B_0', 11: 'A_6', 
        12: 'C_1', 13: 'C_3', 14: 'C_0', 15: 'B_6', 
        16: 'C_2', 17: 'C_5', 18: 'C_6'}
    # 将行为ID转为编码
    code_y, code_y_pred = mapping[y], mapping[y_pred]
    if code_y == code_y_pred: #编码完全相同得分1.0
        return 1.0
    elif code_y.split("_")[0] == code_y_pred.split("_")[0]: #编码仅字母部分相同得分1.0/7
        return 1.0/7
    elif code_y.split("_")[1] == code_y_pred.split("_")[1]: #编码仅数字部分相同得分1.0/3
        return 1.0/3
    else:
        return 0.0
score = sum(acc_combo(y_true, y_pred) for y_true, y_pred in zip(y_cls, proba_y)) / proba_y.shape[0]
print(round(score, 5))

sub.behavior_id=np.argmax(proba_t, axis=1)
print(sub)
from datetime import *
current = datetime.now()
current=current.strftime('%m-%d-%H-%M')
sub.to_csv('data/%s_sub%.5f.csv' %(current,score), index=False)
#生成半监督训练数据
sub["proba"]=np.max(proba_t,axis=1)
sub.to_csv("data/semi_test_8_nobest.csv",index=False)
sub.info()